### 0. Libraries and utilities

In [1]:
import re
import unicodedata
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ccsar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove accents
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenize the text
    tokens = text.split()

    # Remove English stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Join the tokens back into a single string
    text = ' '.join(tokens)

    return text

# Limpiamos el texto
preprocess_text('What can I say, I love this place')

'say love place'

### 1. Preprocessing

In [3]:
# data_path = r'Data\Gungor_2018_VictorianAuthorAttribution_data-train.csv'
# df = pd.read_csv(data_path, encoding='latin-1')

## URL from github repo, load as dataframe
url = 'https://raw.githubusercontent.com/ccsarmientot/text_author_classifier/master/datasets/sample_victorian.parquet'
url = 'datasets/sample_victorian.parquet'
df = pd.read_parquet(url)
df = df.sample(1250)

print(f'Shape of dataframe: {df.shape}')
df.head(5)

Shape of dataframe: (1250, 2)


,text,author
41611,excuse our lives were in danger by these savag...,38
36824,evidently expected instant death and held up h...,35
7832,united ever for your welfare ever for your ben...,8
51458,ages here and there might be seen a rude perch...,48
47014,development here having first seen the profit ...,43


In [4]:
avg_chr = np.mean(df['text'].apply(len))
f'Cantidad promedio de caracteres por texto: {avg_chr:,.2f}'

'Cantidad promedio de caracteres por texto: 4,942.12'

In [5]:
avg_chr = np.mean(df['text'].apply(lambda x: len(x.split(' '))))
f'Cantidad promedio de palabras por texto: {avg_chr:,.2f}'

'Cantidad promedio de palabras por texto: 1,001.00'

In [6]:
## Se identifica un desbalance de clases:
df['author'].value_counts()

author
8     171
26    103
14     64
39     61
33     59
45     55
21     44
48     42
37     41
4      38
19     36
43     34
15     33
18     31
25     28
42     26
1      24
38     24
41     22
30     22
32     21
17     20
28     18
10     17
34     17
29     15
9      15
50     15
35     14
11     14
20     13
12     12
44     12
36     12
22     10
46      9
23      9
40      9
2       8
6       7
13      6
27      6
24      5
16      5
3       3
Name: count, dtype: int64

In [7]:
# ## Getting sample fo 
# df_sample = df.sample(10_000)
# n_authors = df_sample['author'].nunique()
# print(f'Authors in df_sample: {n_authors}')
# df_sample.head(5)

## 2. Modelling

In [8]:
# Importamos librerias
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler

results = []
X = df['text']
y = df['author']

# Dividimos los datos en entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 2.1 Logistic Regression

In [9]:
# Creacion del pipeline del modelo inicial
model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("logit", LogisticRegression())
])

#### 2.1.1 Primera iteración grid search

In [10]:
from sklearn.model_selection import GridSearchCV

# Definimos los parámetros a explorar
param_grid = {
    'tfidf__max_features': [500, 1000],
    'logit__C': [0.1, 1, 10],
}

# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


c:\Users\ccsar\miniconda3\envs\env_nlp\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


[CV] END ..............logit__C=0.1, tfidf__max_features=500; total time=   4.6s
[CV] END ..............logit__C=0.1, tfidf__max_features=500; total time=   3.9s
[CV] END ..............logit__C=0.1, tfidf__max_features=500; total time=   4.3s
[CV] END ..............logit__C=0.1, tfidf__max_features=500; total time=   3.9s
[CV] END ..............logit__C=0.1, tfidf__max_features=500; total time=   5.5s
[CV] END .............logit__C=0.1, tfidf__max_features=1000; total time=   4.2s
[CV] END .............logit__C=0.1, tfidf__max_features=1000; total time=   6.6s
[CV] END .............logit__C=0.1, tfidf__max_features=1000; total time=   4.6s
[CV] END .............logit__C=0.1, tfidf__max_features=1000; total time=   4.6s
[CV] END .............logit__C=0.1, tfidf__max_features=1000; total time=   4.9s
[CV] END ................logit__C=1, tfidf__max_features=500; total time=  20.0s
[CV] END ................logit__C=1, tfidf__max_features=500; total time=  19.3s
[CV] END ................log

#### 2.1.2 Segunda iteración grid search

In [13]:
# Definimos los parámetros a explorar
param_grid = {
    'tfidf__max_features': [1000],
    'logit__C': [9, 10, 11],
}

# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


c:\Users\ccsar\miniconda3\envs\env_nlp\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


[CV] END ..............logit__C=0.1, tfidf__max_features=500; total time=   6.7s
[CV] END ..............logit__C=0.1, tfidf__max_features=500; total time=   6.3s
[CV] END ..............logit__C=0.1, tfidf__max_features=500; total time=   4.1s
[CV] END ..............logit__C=0.1, tfidf__max_features=500; total time=   3.9s
[CV] END ..............logit__C=0.1, tfidf__max_features=500; total time=   4.1s
[CV] END .............logit__C=0.1, tfidf__max_features=1000; total time=   3.9s
[CV] END .............logit__C=0.1, tfidf__max_features=1000; total time=   4.1s
[CV] END .............logit__C=0.1, tfidf__max_features=1000; total time=   6.3s
[CV] END .............logit__C=0.1, tfidf__max_features=1000; total time=   5.7s
[CV] END .............logit__C=0.1, tfidf__max_features=1000; total time=   4.0s
[CV] END ................logit__C=1, tfidf__max_features=500; total time=   6.9s
[CV] END ................logit__C=1, tfidf__max_features=500; total time=   5.9s
[CV] END ................log

#### 2.1.3 Modelo final

In [ ]:
# Creacion del pipeline
final_model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(max_features=500, preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("logit", LogisticRegression(C=0.1))
])

In [15]:
# Ajustamos el modelo
final_model.fit(X_train, y_train)

In [ ]:
# Medimos el accuracy del modelo
accuracy = final_model.score(X_test, y_test)
accuracy

In [ ]:
# Se predicen las clases para test
y_pred = model.predict(X_test)

#### 2.1.4 Save metrics

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
p, r, f1, s = precision_recall_fscore_support(y_test, y_pred, average='weighted')
model_name = 'Logistic_regresion'
results.append([model_name, p, r, f1, s, tn, fp, fn, tp])

### 2.2 Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Creacion del pipeline del modelo inicial
model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("naive", MultinomialNB())
])

#### 2.2.1 Primera iteración grid search

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definimos los parámetros a explorar
param_grid = {
    'tfidf__max_features': [500, 1000],
    'naive__alpha': [0.1, 1, 10],
}

# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

#### 2.2.2 Segunda iteración grid search

In [ ]:
# Definimos los parámetros a explorar
param_grid = {
    'tfidf__max_features': [500, 1000],
    'naive__alpha': [0.1, 1, 10],
}

# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

#### 2.2.3 Modelo final

In [ ]:
# Creacion del pipeline
final_model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(max_features=500, preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("naive", MultinomialNB(alpha=0.1))
])

In [ ]:
# Ajustamos el modelo
final_model.fit(X_train, y_train)

In [ ]:
# Medimos el accuracy del modelo
accuracy = final_model.score(X_test, y_test)
accuracy

In [ ]:
# Se predicen las clases para test
y_pred = model.predict(X_test)

#### 2.2.4 Save metrics

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
p, r, f1, s = precision_recall_fscore_support(y_test, y_pred, average='weighted')
model_name = 'NaiveBayes'
results.append([model_name, p, r, f1, s, tn, fp, fn, tp])

### 2.3 KNeighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Creacion del pipeline del modelo inicial
model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("kneig", KNeighborsClassifier())
])

#### 2.3.1 Primera iteración grid search

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definimos los parámetros a explorar
param_grid = {
    'tfidf__max_features': [500, 1000],
    'kneig__n_neighbors': [3, 5],
    'kneig__metric': ['euclidean', 'manhattan', 'cosine']
}

# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

#### 2.3.2 Segunda iteración grid search

In [ ]:
# Definimos los parámetros a explorar
param_grid = {
    'tfidf__max_features': [500, 1000],
    'kneig__n_neighbors': [3, 5],
    'kneig__metric': ['euclidean', 'manhattan', 'cosine']
}


# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

#### 2.3.3 Modelo final

In [ ]:
# Creacion del pipeline
final_model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(max_features=500, preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("kneig", KNeighborsClassifier(n_neighbors=3, metric='euclidean'))
])

In [ ]:
# Ajustamos el modelo
final_model.fit(X_train, y_train)

In [ ]:
# Medimos el accuracy del modelo
accuracy = final_model.score(X_test, y_test)
accuracy

In [ ]:
# Se predicen las clases para test
y_pred = model.predict(X_test)

#### 2.3.4 Save metrics

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
p, r, f1, s = precision_recall_fscore_support(y_test, y_pred, average='weighted')
model_name = 'K-neighbors'
results.append([model_name, p, r, f1, s, tn, fp, fn, tp])

### 2.4 Arboles de decisión

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Creacion del pipeline del modelo inicial
model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("tree", DecisionTreeClassifier())
])

#### 2.4.1 Primera iteración grid search

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definimos los parámetros a explorar
param_grid = {
    'tfidf__max_features': [500, 1000],
    'tree__criterion': ['gini', 'entropy'],
    'tree__max_depth': [5, 10],
    'tree__min_samples_split': [5, 10],
    'tree__min_samples_leaf': [1, 3]
}

# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

#### 2.4.2 Segunda iteración grid search

In [ ]:
# Definimos los parámetros a explorar
# Definimos los parámetros a explorar
param_grid = {
    'tfidf__max_features': [500, 1000],
    'tree__criterion': ['gini', 'entropy'],
    'tree__max_depth': [5, 10],
    'tree__min_samples_split': [5, 10],
    'tree__min_samples_leaf': [1, 3]
}



# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

#### 2.3.3 Modelo final

In [ ]:
# Creacion del pipeline
final_model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(max_features=500, preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("tree", DecisionTreeClassifier(max_depth=5, min_samples_split=5, min_samples_leaf=3))
])

In [ ]:
# Ajustamos el modelo
final_model.fit(X_train, y_train)

In [ ]:
# Medimos el accuracy del modelo
accuracy = final_model.score(X_test, y_test)
accuracy

In [ ]:
# Se predicen las clases para test
y_pred = model.predict(X_test)

#### 2.3.4 Save metrics

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
p, r, f1, s = precision_recall_fscore_support(y_test, y_pred, average='weighted')
model_name = 'Decision Trees'
results.append([model_name, p, r, f1, s, tn, fp, fn, tp])

### 2.5 Support Vector Machine

In [ ]:
from sklearn.svm import SVC

# Creacion del pipeline del modelo inicial
model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("SVC", SVC())
])

#### 2.5.1 Primera iteración grid search

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definimos los parámetros a explorar
param_grid = {
    'tfidf__max_features': [500, 1000],
    'SVC__C': [0.1, 1, 10, 100],
    'SVC__kernel': ['linear', 'rbf'],
    'SVC__gamma': [0.1, 1, 10]
}

# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

#### 2.5.2 Segunda iteración grid search

In [ ]:
# Definimos los parámetros a explorar
# Definimos los parámetros a explorar
param_grid = {
    'tfidf__max_features': [500, 1000],
    'SVC__C': [0.1, 1, 10, 100],
    'SVC__kernel': ['linear', 'rbf'],
    'SVC__gamma': [0.1, 1, 10]
}


# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

#### 2.5.3 Modelo final

In [ ]:
# Creacion del pipeline
final_model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(max_features=500, preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("SVC", SVC(C=0.1, kernel='linear', gamma=0.1))
])

In [ ]:
# Ajustamos el modelo
final_model.fit(X_train, y_train)

In [ ]:
# Medimos el accuracy del modelo
accuracy = final_model.score(X_test, y_test)
accuracy

In [ ]:
# Se predicen las clases para test
y_pred = model.predict(X_test)

#### 2.5.4 Save metrics

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
p, r, f1, s = precision_recall_fscore_support(y_test, y_pred, average='weighted')
model_name = 'SVC'
results.append([model_name, p, r, f1, s, tn, fp, fn, tp])

### 2.6 Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Creacion del pipeline del modelo inicial
model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("rforest", RandomForestClassifier())
])

#### 2.3.1 Primera iteración grid search

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definimos los parámetros a explorar
param_grid = {
    'tfidf__max_features': [500, 1000],
    'rforest__n_estimators': [100, 200],
    'rforest__max_depth': [5, 10],
    'rforest__min_samples_split': [2, 5],
    'rforest__min_samples_leaf': [2, 4]
}

# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

#### 2.3.2 Segunda iteración grid search

In [ ]:
# Definimos los parámetros a explorar
param_grid = {
    'tfidf__max_features': [500, 1000],
    'rforest__n_estimators': [100, 200],
    'rforest__max_depth': [5, 10],
    'rforest__min_samples_split': [2, 5],
    'rforest__min_samples_leaf': [2, 4]
}


# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

#### 2.3.3 Modelo final

In [ ]:
# Creacion del pipeline
final_model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(max_features=500, preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("rforest", RandomForestClassifier(n_estimators=100, max_depth=5, min_samples_split=2, min_samples_leaf=2))
])

In [ ]:
# Ajustamos el modelo
final_model.fit(X_train, y_train)

In [ ]:
# Medimos el accuracy del modelo
accuracy = final_model.score(X_test, y_test)
accuracy

In [ ]:
# Se predicen las clases para test
y_pred = model.predict(X_test)

#### 2.3.4 Save metrics

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
p, r, f1, s = precision_recall_fscore_support(y_test, y_pred, average='weighted')
model_name = 'Random Forest'
results.append([model_name, p, r, f1, s, tn, fp, fn, tp])

### 2.7 XGBoost

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

# Creacion del pipeline del modelo inicial
model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("XGB", xgb())
])

#### 2.7.1 Primera iteración grid search

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'tfidf__max_features': [500, 1000],
    'XGB__max_depth': [3, 5],
    'XGB__learning_rate': [0.1, 0.3],
    'XGB__subsample': [0.8, 1.0],
    'XGB__colsample_bytree': [0.8, 1.0]
}

# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

#### 2.7.2 Segunda iteración grid search

In [ ]:
# Definimos los parámetros a explorar
param_grid = {
    'tfidf__max_features': [500, 1000],
    'XGB__max_depth': [3, 5],
    'XGB__learning_rate': [0.1, 0.3],
    'XGB__subsample': [0.8, 1.0],
    'XGB__colsample_bytree': [0.8, 1.0]
}

# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

#### 2.7.3 Modelo final

In [ ]:
# Creacion del pipeline
final_model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(max_features=500, preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("XGB", xgb(max_depth=3, learning_rate=0.1, subsample=0.8, colsample_bytree=0-8))
])

In [ ]:
# Ajustamos el modelo
final_model.fit(X_train, y_train)

In [ ]:
# Medimos el accuracy del modelo
accuracy = final_model.score(X_test, y_test)
accuracy

In [ ]:
# Se predicen las clases para test
y_pred = model.predict(X_test)

#### 2.7.4 Save metrics

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
p, r, f1, s = precision_recall_fscore_support(y_test, y_pred, average='weighted')
model_name = 'XGBoost'
results.append([model_name, p, r, f1, s, tn, fp, fn, tp])

### 2.8 MLP Classifier (Multi-Layer Perceptron)

In [ ]:
from sklearn.neural_network import MLPClassifier

# Creacion del pipeline del modelo inicial
model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("MLP", MLPClassifier())
])

#### 2.8.1 Primera iteración grid search

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'tfidf__max_features': [500, 1000],
    'MLP__hidden_layer_sizes': [(100, 50), (50, 50)],
    'MLP__activation': ['relu', 'tanh'],
    'MLP__solver': ['adam', 'sgd'],
    'MLP__alpha': [0.001, 0.01],
    'MLP__learning_rate': ['constant', 'adaptive']
}

# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

#### 2.8.2 Segunda iteración grid search

In [ ]:
# Definimos los parámetros a explorar
param_grid = {
    'tfidf__max_features': [500, 1000],
    'MLP__hidden_layer_sizes': [(100, 50), (50, 50)],
    'MLP__activation': ['relu', 'tanh'],
    'MLP__solver': ['adam', 'sgd'],
    'MLP__alpha': [0.001, 0.01],
    'MLP__learning_rate': ['constant', 'adaptive']
}


# Creamos el objeto GridSearchCV
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          verbose=2)

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtenemos los mejores parámetros
best_params = grid_search.best_params_
print(best_params)

#### 2.8.3 Modelo final

In [ ]:
# Creacion del pipeline
final_model = Pipeline(steps=[
    ## Selección de los features
    ("tfidf", TfidfVectorizer(max_features=500, preprocessor=preprocess_text)),
    ## Se tiene un desbalance de clases, entonces se hace un oversampling
    ('oversample', RandomOverSampler()),
    ## Se aplica el modelo
    ("MLP", MLPClassifier(hidden_layer_sizes=(50, 50), activation='relu', solver='sgd', alpha=0.01, learning_rate='adaptative'))
])

In [ ]:
# Ajustamos el modelo
final_model.fit(X_train, y_train)

In [ ]:
# Medimos el accuracy del modelo
accuracy = final_model.score(X_test, y_test)
accuracy

In [ ]:
# Se predicen las clases para test
y_pred = model.predict(X_test)

#### 2.8.4 Save metrics

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
p, r, f1, s = precision_recall_fscore_support(y_test, y_pred, average='weighted')
model_name = 'MLP'
results.append([model_name, p, r, f1, s, tn, fp, fn, tp])